In [1]:
# Native libraries
import os
import math
# Essential Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from fredapi import Fred
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
# Algorithms
from statistics import mean, stdev, variance
from minisom import MiniSom
from tslearn.barycenters import dtw_barycenter_averaging
from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import KMeans, MeanShift, estimate_bandwidth
from collections import Counter
from sklearn.decomposition import PCA
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


/home/dwight/miniconda3/lib/python3.8/site-packages/tslearn/clustering/kmeans.py:16: UserWarning: Scikit-learn <0.24 will be deprecated in a future release of tslearn
  warnings.warn(


In [2]:
csv = pd.read_csv("monthly_data.csv")


In [3]:
fred = Fred(api_key='3c6ba58d26525f17af95af4fabed24be')


In [4]:
csv2 = csv[50:-20]
csv2 = csv2.drop(['SMU06419406562200001.1', 'SMU21000006056170001SA.1'], axis=1)

In [32]:
data = csv2.dropna(axis="columns")[8::3].reset_index().drop(["index", "Unnamed: 0"], axis=1)
data

,UNRATE,CPIAUCSL,FEDFUNDS,CSUSHPINSA,M2SL,M1SL,PSAVERT,PAYEMS,INDPRO,TB3MS,...,ESTPIEAMP01GPM,BROSERPA158MFRBDAL,LARTTULA158MFRBDAL,LAUCN310910000000004,LAUCN271470000000004,LFWATTFECAM647N,LFWA55FECAM647S,LFWA55FEKRM647S,LFWA55MAAUM647N,LFWA55MAAUM647S
0,5.4,191.700,1.93,157.497,6399.4,1374.2,4.0,132503.0,97.6137,2.07,...,-0.202429,-1.713722,2.689583,12.0,712.0,12971900.0,1725500.0,2.143286e+06,1063251.0,1.062949e+06
1,5.4,192.400,2.50,161.924,6432.5,1371.1,3.2,133032.0,99.4639,2.54,...,-0.201410,1.561165,16.658304,25.0,983.0,13010400.0,1745200.0,2.138804e+06,1072885.0,1.073819e+06
2,5.1,193.600,3.00,169.544,6473.1,1366.0,3.5,133690.0,99.6033,2.84,...,-0.100604,1.090706,-4.866917,11.0,747.0,13057400.0,1763600.0,2.166473e+06,1081863.0,1.081815e+06
3,4.9,196.100,3.50,175.922,6569.8,1377.8,2.6,134498.0,99.9435,3.44,...,1.206030,9.230125,-3.218818,9.0,726.0,13110300.0,1782100.0,2.175409e+06,1090390.0,1.090723e+06
4,5.0,198.100,4.00,179.675,6654.5,1376.1,3.5,134993.0,100.3216,3.88,...,0.000000,2.638735,6.844361,8.0,713.0,13152700.0,1798800.0,2.175877e+06,1099186.0,1.098971e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,3.7,256.118,2.13,211.802,14938.8,3844.0,7.3,151108.0,109.8543,1.95,...,-0.223881,2.935424,-1.687233,13.0,642.0,15569800.0,2620100.0,3.998115e+06,1437108.0,1.438242e+06
60,3.6,257.989,1.55,212.222,15254.4,3955.6,7.5,151758.0,110.0388,1.54,...,0.150376,5.135172,-4.344529,11.0,505.0,15624700.0,2625300.0,4.015310e+06,1444387.0,1.443890e+06
61,3.5,258.824,1.58,213.295,15473.4,4027.6,8.3,152523.0,109.2966,1.52,...,-0.452489,4.145019,2.584733,13.0,752.0,15663700.0,2629400.0,4.020187e+06,1450121.0,1.449740e+06
62,13.3,255.942,0.05,218.569,17893.0,16275.9,24.7,132994.0,92.0613,0.13,...,-0.152555,58.708814,23.355997,10.0,1953.0,15693800.0,2633500.0,4.032882e+06,1453775.0,1.453747e+06


In [6]:
target = pd.read_csv("micron.csv").drop("date", axis=1)[::-1].reset_index().drop("index", axis=1)
target

,sales
0,1308
1,1054
2,1258
3,1362
4,1225
...,...
60,4797
61,5438
62,6056
63,5773


In [7]:
full = data.join(target)

In [8]:
names = full
scaler = MinMaxScaler()
full = MinMaxScaler().fit_transform(full)
som_x = som_y = math.ceil(math.sqrt(math.sqrt(len(full))))
full = np.transpose(full)

In [9]:
since = time.time()
som_x = som_y = math.ceil(math.sqrt(math.sqrt(len(full))))
# I didn't see its significance but to make the map square,
# I calculated square root of map size which is 
# the square root of the number of series
# for the row and column counts of som

som = MiniSom(som_x, som_y,len(full[0]), sigma=0.3, learning_rate = 0.1)

som.random_weights_init(full)
som.train(full, 50000)
time_elapsed = time.time() - since
print(f"SOM clustering executed in {time_elapsed // 60}m {time_elapsed % 60}s ")

SOM clustering executed in 0.0m 5.419386863708496s 


In [10]:
since = time.time()
pca = PCA(n_components=2)

mySeries_transformed = pca.fit_transform(full)

time_elapsed = time.time() - since
print(f"PCA clustering executed in {time_elapsed // 60}m {time_elapsed % 60}s ")

PCA clustering executed in 0.0m 0.2759852409362793s 


In [11]:
since = time.time()
cluster_count = math.ceil(math.sqrt(len(full))) 
print(cluster_count)
kmeans = KMeans(n_clusters=60,max_iter=5000)

labels = kmeans.fit_predict(mySeries_transformed)
time_elapsed = time.time() - since
print(f"Kmeans on PCA clustering executed in {time_elapsed // 60}m {time_elapsed % 60}s ")

297
Kmeans on PCA clustering executed in 0.0m 41.30145812034607s 


In [12]:
cluster_map = []
i = 0
cluster_x = []
cluster_dict = {}
for idx in names:
    winner_node = som.winner(full[i])
    name = f"Cluster {winner_node[0] * som_y + winner_node[1] + 1}"
    cluster_map.append((idx, name))

    cluster_x.append(name)
    i += 1

cluster_x = Counter(cluster_x)
clusters = pd.DataFrame(cluster_map, columns=["Name", "Cluster"]).sort_values(by="Cluster").set_index("Name")

In [13]:
cluster = clusters.loc["sales"].values
value = clusters["Cluster"].values
index = list(np.where(value ==cluster)[0])
names = list(clusters.index.values[index])
names.remove("sales")
names

['IMP5200',
 'IPG311N',
 'LAUCN080870000000005',
 'SMU15279806500000001',
 'SMU15279806500000001SA',
 'SMU17000003231100001',
 'SMU18000003200000008',
 'SMU01000003200000030SA',
 'CP0932HUM086NEST',
 'SMU01000003200000007SA',
 'SMU01000003200000007',
 'EXPMANNH',
 'SMU29000005000000008',
 'IPG311S',
 'CP0931DEM086NEST',
 'SMU25731046055000001SA',
 'SMU18000003200000008SA',
 'WPS011101',
 'CP0511MTM086NEST',
 'SMU49362609092000001SA',
 'SMU45348204300000001SA',
 'SMU44772004300000001SA',
 'IRLLOCOXGORSTSAM',
 'APMAINNOM086NEST',
 'SMU55333403000000008SA',
 'SMU26000002023830030',
 'SMU27000004245200030',
 'EXPMANND',
 'SMU29000005000000008SA',
 'SMU29000004100000008',
 'SMU12367406055000001',
 'SMU34000003231100001',
 'SMU55333403000000030',
 'SMU29000004100000008SA',
 'SMU27000003231160001SA',
 'HANF206TRADN',
 'SMU42000002023700001SA',
 'SMU45348204300000001',
 'SMU25731046055000001',
 'SMU29000005000000030',
 'SMU27000004245200030SA',
 'SMU53000004349300001',
 'SMU26000002023830007SA

In [ ]:
corrs = {}
for name in data:
    x = data[name]
    
    corrs[abs(x.corr(target["sales"]))] = name

In [ ]:
names = []
for num in sorted(corrs)[:-40:-1]:
    names.append(corrs[num])
for num in sorted(corrs)[:10:1]:
    names.append(corrs[num])

In [ ]:
data = data[names]

In [ ]:
full = data.join(target)

In [ ]:
full

In [ ]:
X, y = full.iloc[:,:-1],full.iloc[:,-1]


In [ ]:
data_dmatrix = xgb.DMatrix(data=X,label=y)



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3,learning_rate = 0.2,
                max_depth = 10, alpha = 5,n_estimators = 300, tree_method="gpu_hist")

In [ ]:
xg_reg.fit(X_train,y_train)


In [ ]:
preds = xg_reg.predict(X_test)


In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [ ]:
clf = Lasso(alpha=0.4, max_iter=200000)
clf.fit(X_train, y_train)

In [ ]:
preds = clf.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [43]:
values = {}
for name in names:
    d = fred.get_series(name, observation_start="2/1/2021", observation_end = "2/1/2021")
    values[name] = d

ValueError: Bad Request.  Invalid value for variable series_id.  Series IDs should be 25 or less alphanumeric characters.

In [ ]:
values["sales"] = 6236
test = pd.DataFrame(values)


In [ ]:
test_data = torch.from_numpy(test.to_numpy())
test_data2 = torch.from_numpy(full[:-3:-1].to_numpy())
test_data = torch.cat([test_data2, test_data]).unsqueeze(0).float()

In [ ]:
model = Model()
model.load_state_dict(torch.load("model.pt"))
model(test_data)

In [ ]:
xg_reg.predict(test)

In [101]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
     

        self.output_block = nn.Sequential(nn.Conv1d(3, 16, 2, 1, 0), nn.ReLU(), 
                                          nn.Conv1d(16, 32, 2, 1, 0), nn.ReLU(), 
                                          nn.Conv1d(32, 32, 2, 1, 0), nn.ReLU(), 
                                          nn.Conv1d(32, 16, 2, 1, 0), nn.ReLU())
        self.fc = nn.Sequential(nn.LazyLinear(512), nn.ReLU(), nn.Linear(512, 256), nn.ReLU(), nn.Linear(256, 1))

    def forward(self, x):
        net = self.output_block(x)
        net = net.view(net.shape[0], -1)
        net = self.fc(net)
        
        return net


In [102]:
target[3:]

,sales
3,1362
4,1225
5,1312
6,1373
7,1530
...,...
60,4797
61,5438
62,6056
63,5773


In [103]:
class dataset(Dataset):
    def __init__(self, df, target):
        data = df[1:].reset_index(drop=True)
        sales = target[:-1].reset_index(drop=True)
        self.data = data.join(sales)
        self.target = target[3:].to_numpy()

    def __len__(self):
        return len(self.data)-3
    def __getitem__(self, idx):
        data = self.data.iloc[idx:idx+3].to_numpy()
        target = self.target[idx]
        return torch.tensor(data).float(), torch.tensor(target).float()
        
    

In [104]:
idx = int(len(data) * 0.8)
train, test = data[:idx], data[idx:]
traint, testt = target[:idx], target[idx:]
trainset = dataset(train, traint)
testset = dataset(test, testt)
train_loader = DataLoader(trainset, batch_size=8, shuffle=True)
test_loader = DataLoader(testset, batch_size=4, shuffle=True)

In [109]:
model = Model()
model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.003)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=25, factor=0.5, min_lr=0.000001)
def Train(epochs, model, train_loader):
    valid_loss_min = np.Inf
    min_rmse = np.Inf
    for epoch in range(epochs):
        train_loss = 0.0
        correct = 0.0
        total = 0.0
        valid_loss = 0.0
        rmse = 0.0
        model.train()
        for batch_idx, (images, labels) in enumerate(train_loader):
       
            images = images.cuda()
            labels = labels.cuda()
            output = model(images)
        
            loss = torch.sqrt(criterion(output,labels))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
        model.eval()
        for batch_idx, (images, labels) in enumerate(test_loader):
       
            images = images.cuda()
            labels = labels.cuda()
            output = model(images)
        
            loss = torch.sqrt(criterion(output,labels))
          
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
                
        scheduler.step(valid_loss)
        print(f"Train Loss: {train_loss.item()}")
        print(f"Valid Loss: {valid_loss.item()}")
        if valid_loss.item() < valid_loss_min:
            valid_loss_min = valid_loss.item()
            torch.save(model.state_dict(), 'model.pt')
    print(f"Min RMSE: {valid_loss_min}")

In [110]:
Train(1000, model, train_loader)




Train Loss: 1909.0980224609375
Valid Loss: 4860.9033203125
Train Loss: 1539.817138671875
Valid Loss: 3125.787109375
Train Loss: 1479.136474609375
Valid Loss: 4563.7060546875
Train Loss: 1701.6622314453125
Valid Loss: 4160.05810546875
Train Loss: 1390.8046875
Valid Loss: 3421.848876953125
Train Loss: 1422.8558349609375
Valid Loss: 4416.681640625
Train Loss: 1272.353515625
Valid Loss: 3476.4189453125
Train Loss: 1139.6719970703125
Valid Loss: 4672.57568359375
Train Loss: 1451.4915771484375
Valid Loss: 4324.1904296875
Train Loss: 1283.2880859375
Valid Loss: 2497.315673828125
Train Loss: 1174.651123046875
Valid Loss: 4050.953125
Train Loss: 1246.9923095703125
Valid Loss: 2587.421875
Train Loss: 1279.2352294921875
Valid Loss: 4798.984375
Train Loss: 1120.805908203125
Valid Loss: 2230.7822265625
Train Loss: 1052.0771484375
Valid Loss: 4384.22607421875
Train Loss: 1016.8716430664062
Valid Loss: 1846.4305419921875
Train Loss: 1020.7130126953125
Valid Loss: 2266.962890625
Train Loss: 746.688842

Train Loss: 123.72540283203125
Valid Loss: 863.6663818359375
Train Loss: 123.79112243652344
Valid Loss: 800.5851440429688
Train Loss: 123.38768005371094
Valid Loss: 1067.212890625
Train Loss: 130.05218505859375
Valid Loss: 677.6962890625
Train Loss: 125.4349136352539
Valid Loss: 1059.8778076171875
Train Loss: 118.3251953125
Valid Loss: 796.4824829101562
Train Loss: 126.9022216796875
Valid Loss: 1057.194580078125
Train Loss: 123.50830841064453
Valid Loss: 877.8069458007812
Train Loss: 120.07618713378906
Valid Loss: 755.4863891601562
Train Loss: 122.2348403930664
Valid Loss: 913.6035766601562
Train Loss: 119.61326599121094
Valid Loss: 923.7738647460938
Train Loss: 123.96630859375
Valid Loss: 778.0222778320312
Train Loss: 117.31217956542969
Valid Loss: 1075.2857666015625
Train Loss: 115.39347076416016
Valid Loss: 890.9816284179688
Train Loss: 119.67304992675781
Valid Loss: 781.2518920898438
Train Loss: 120.36311340332031
Valid Loss: 811.6633911132812
Train Loss: 118.59669494628906
Valid L

Train Loss: 116.24423217773438
Valid Loss: 913.983642578125
Train Loss: 112.36417388916016
Valid Loss: 801.04296875
Train Loss: 110.4356918334961
Valid Loss: 720.58203125
Train Loss: 111.31568908691406
Valid Loss: 798.945068359375
Train Loss: 112.75006866455078
Valid Loss: 773.8458251953125
Train Loss: 111.91371154785156
Valid Loss: 753.4432373046875
Train Loss: 116.84313201904297
Valid Loss: 847.7421875
Train Loss: 116.8740005493164
Valid Loss: 799.3917846679688
Train Loss: 112.21647644042969
Valid Loss: 745.767578125
Train Loss: 109.15863800048828
Valid Loss: 825.7821044921875
Train Loss: 111.95008850097656
Valid Loss: 909.0669555664062
Train Loss: 109.27018737792969
Valid Loss: 935.7445678710938
Train Loss: 113.50057983398438
Valid Loss: 753.9373168945312
Train Loss: 111.62117767333984
Valid Loss: 732.081298828125
Train Loss: 112.65349578857422
Valid Loss: 983.2135620117188
Train Loss: 115.01866149902344
Valid Loss: 725.5088500976562
Train Loss: 111.79288482666016
Valid Loss: 992.27

Train Loss: 110.80066680908203
Valid Loss: 914.0237426757812
Train Loss: 114.0172119140625
Valid Loss: 827.7887573242188
Train Loss: 107.54045104980469
Valid Loss: 726.353271484375
Train Loss: 111.79805755615234
Valid Loss: 994.3298950195312
Train Loss: 111.98477935791016
Valid Loss: 936.2490844726562
Train Loss: 110.94713592529297
Valid Loss: 890.6657104492188
Train Loss: 105.63764190673828
Valid Loss: 718.7747802734375
Train Loss: 111.44446563720703
Valid Loss: 874.5654907226562
Train Loss: 111.95745086669922
Valid Loss: 784.176513671875
Train Loss: 115.40194702148438
Valid Loss: 802.4317016601562
Train Loss: 113.77745819091797
Valid Loss: 986.9462280273438
Train Loss: 114.40467071533203
Valid Loss: 794.6248779296875
Train Loss: 115.20021057128906
Valid Loss: 789.280029296875
Train Loss: 111.06356811523438
Valid Loss: 893.263916015625
Train Loss: 115.75180053710938
Valid Loss: 705.5333862304688
Train Loss: 114.4776382446289
Valid Loss: 925.6724853515625
Train Loss: 110.41936492919922

Train Loss: 111.22925567626953
Valid Loss: 738.6792602539062
Train Loss: 113.08966064453125
Valid Loss: 863.8427124023438
Train Loss: 111.64733123779297
Valid Loss: 775.7091064453125
Train Loss: 114.54255676269531
Valid Loss: 996.1203002929688
Train Loss: 105.15550231933594
Valid Loss: 732.5252075195312
Train Loss: 106.81468963623047
Valid Loss: 791.4558715820312
Train Loss: 107.93365478515625
Valid Loss: 870.6603393554688
Train Loss: 106.96981811523438
Valid Loss: 817.143310546875
Train Loss: 111.4213638305664
Valid Loss: 747.0218505859375
Train Loss: 108.58367156982422
Valid Loss: 875.02294921875
Train Loss: 115.84741973876953
Valid Loss: 771.148681640625
Train Loss: 107.5038833618164
Valid Loss: 830.148681640625
Train Loss: 113.87451171875
Valid Loss: 925.7423706054688
Train Loss: 108.75664520263672
Valid Loss: 797.025634765625
Train Loss: 110.99972534179688
Valid Loss: 870.4384155273438
Train Loss: 116.50851440429688
Valid Loss: 726.2362670898438
Train Loss: 112.69031524658203
Vali

Train Loss: 111.32588958740234
Valid Loss: 798.2139282226562
Train Loss: 110.87149810791016
Valid Loss: 782.9210205078125
Train Loss: 112.72086334228516
Valid Loss: 997.3516235351562
Train Loss: 107.16419982910156
Valid Loss: 769.755126953125
Train Loss: 108.31005096435547
Valid Loss: 781.93994140625
Train Loss: 110.4136962890625
Valid Loss: 832.067138671875
Train Loss: 113.20555877685547
Valid Loss: 1107.60498046875
Train Loss: 112.46891784667969
Valid Loss: 935.2734985351562
Train Loss: 114.17710876464844
Valid Loss: 798.8885498046875
Train Loss: 112.98951721191406
Valid Loss: 834.3778686523438
Train Loss: 107.28947448730469
Valid Loss: 1122.1116943359375
Train Loss: 108.08219909667969
Valid Loss: 725.94775390625
Train Loss: 110.81704711914062
Valid Loss: 919.59814453125
Train Loss: 109.20233154296875
Valid Loss: 1102.938232421875
Train Loss: 115.06752014160156
Valid Loss: 888.7695922851562
Train Loss: 115.39398956298828
Valid Loss: 831.8958740234375
Train Loss: 111.03118133544922
Va

Train Loss: 111.08951568603516
Valid Loss: 1100.107421875
Train Loss: 114.69695281982422
Valid Loss: 744.1094360351562
Train Loss: 116.25482177734375
Valid Loss: 994.825927734375
Train Loss: 111.68946075439453
Valid Loss: 730.3470458984375
Train Loss: 109.29320526123047
Valid Loss: 998.4586791992188
Train Loss: 116.57160949707031
Valid Loss: 939.901123046875
Train Loss: 107.59638214111328
Valid Loss: 761.652099609375
Train Loss: 110.42915344238281
Valid Loss: 835.6305541992188
Train Loss: 111.99349212646484
Valid Loss: 776.5693969726562
Train Loss: 114.291259765625
Valid Loss: 792.171142578125
Train Loss: 109.99340057373047
Valid Loss: 777.686767578125
Train Loss: 110.50584411621094
Valid Loss: 843.9744262695312
Train Loss: 105.75413513183594
Valid Loss: 1122.4970703125
Train Loss: 114.58624267578125
Valid Loss: 999.3917846679688
Train Loss: 116.084716796875
Valid Loss: 925.5983276367188
Train Loss: 105.66157531738281
Valid Loss: 936.361572265625
Train Loss: 107.05984497070312
Valid Lo